# Merging Point Clouds and Retrieving Nested Data 🌲

In this notebook we demonstrate how to merge tree pack point clouds to create a
forest‑wide view and how to extract nested data tables such as whorls, CT metrics
and harvester logs for a particular tree. For the demo dataset we
use CSV files rather than LAZ, but the APIs are the same. See the code comments
for notes on when to use LAZ instead.

In [ ]:
import sqlite3
from singletree.pointcloud.merge import merge_campaign_tree_packs

# For the demo data we created CSV point clouds. Normally you would merge LAZ files.
# Suppose we had processed the ULS campaign into LAZ tree packs, we could merge them as follows:
# merged_asset = merge_campaign_tree_packs(
#     campaign_uid='CAM_ULS_2025-01-02_AREA1',
#     include_ground=True,
#     include_residual=False
# )
# print(merged_asset)

print('(Skipping actual merge in demo because the data are CSV, not LAZ)')


### Retrieving nested tables

Nested tables store details like whorl heights, stem taper points, CT metrics and harvester logs.
To access them, query the appropriate table filtered by `measurement_uid` or `tree_uid`. Here we
retrieve the whorls for tree T001 and the CT metrics for tree T003.

In [ ]:
conn = sqlite3.connect('notebooks/demo_data.gpkg')
c = conn.cursor()

# Find the measurement UID for T001 field inventory
m_uid = c.execute(
    'SELECT measurement_uid FROM measurements WHERE tree_uid = ? AND source_type = ?',
    ('T001', 'field',)
).fetchone()[0]

# Retrieve whorls
whorls = c.execute(
    'SELECT whorl_id, height_from_base_m FROM whorls WHERE measurement_uid = ?',
    (m_uid,)
).fetchall()
print('Whorls for T001:', whorls)

# Retrieve CT metrics for T003
ct = c.execute(
    'SELECT metric_id, knot_count, max_knot_diameter_cm, mean_ring_width_mm, density_kg_m3'
    ' FROM ct_metrics WHERE measurement_uid = ?',
    ('T003_CT_2025-01-05',)
).fetchall()
print('CT metrics for T003:', ct)

conn.close()


You could perform similar queries for `stem_taper`, `harvester_logs` and other nested
tables using the same pattern. With these pieces you can build advanced analyses and
visualisations of your forest data.